In [ ]:
# USAGE
# MR-SLIC_original
from pyspark.context import SparkContext
from skimage.segmentation import mark_boundaries
from skimage import io
import collections as coll
import time
from scipy import ndimage as ndi
from skimage.segmentation import slic
from PIL import Image
from skimage.util import img_as_float
import numpy as np
from skimage.segmentation._slic_master import _slic_cythonM
from skimage.segmentation._slic import _enforce_label_connectivity_cython
from skimage.color import rgb2lab
import os

#목표 세그먼트


OriginSegments = 1000
for partition in range(8,9):
    folder = 'test'
    path = "/home/gangmin/experiment/data/images/"+folder
    file_list = os.listdir(path)
    file_list_img = [file for file in file_list if file.endswith(".jpg")]

    
    # 이미지 조각당 세그먼트 수 
    numSegments = (OriginSegments//partition)
    
    evaluation = []
    
    for img_name in file_list_img:
        
        name = img_name[:-4]

        #원본 이미지 메타데이터(결과 출력용)
        images = Image.open(path+'/'+img_name)
        #원본 이미지 크기
        (img_w,img_h) = images.size
        depth = 1
        #file list
        k=[]
        for i in range(partition):
            k.append(name+'_'+str(i))

        #이미지를 float으로 변환
        images = img_as_float(images)
        #map 함수
        def Map(k):
            #time check
            start_time = time.time()


            #read imagez
            #submit image with job 
            img = Image.open('/home/gangmin/experiment/data/disjoint_images/'+folder+'/'+str(partition)+'/'+ k +'.jpg')

            image = img_as_float(img)
            #슬레이브 노드로 작업 분배
            #각 노드에서 이미지 읽고 작업
            segments,distances = slic(image, n_segments = numSegments, sigma = 5)
            #segments(lable(nesre_segments))데이터와 distance 데이터를 둘다 받는다.


            finish_time = time.time()
            #k 로 1,2,3 숫자 받는다.
            return segments, start_time, finish_time, k[-1]
        #times, segments, distances 를 리턴받는다.
        stime = time.time()
        datas = sc.parallelize(k,partition).map(Map).collect()
        ftime = time.time()



        #원본 이미지 메타데이터로 dimension 전송
        dimension = [depth,img_h,img_w]
        dimension = np.ascontiguousarray(dimension)


        #segments, distances 리스트 만들기
        tempindex = 0
        for i in datas:
            if tempindex is 0:
                seg_array = i[0]
            else:
                seg_array = np.r_[seg_array,i[0]]
            tempindex +=1
       
        labels = seg_array[np.newaxis,:]
        
        
        
        #label 결속 처리
        
        segment_size = depth * img_h * img_w / OriginSegments
        min_size = int(0.5 * segment_size)
        max_size = int(3 * segment_size)
        labels = _enforce_label_connectivity_cython(np.ascontiguousarray(labels),min_size,max_size)

        
        
        fftime = time.time()



        #time check
        import pandas as pd
        data = pd.DataFrame(columns=['node','time'])
        number = 0
        node = 1
        datat = pd.DataFrame(columns=['node','time'])
        pd.options.display.float_format = '{:.6f}'.format
        for i in datas:
            datat.loc[number]=['node '+str(node),i[2]-i[1]]

            data.loc[number]=['node '+str(node)+' start',i[1]]
            number += 1
            data.loc[number]=['node '+str(node)+' finish',i[2]]
            number += 1
            node+=1



        '''
        #print
        data = data.sort_values(by=['time'], axis=0)
        print('##########################################################')
        #slave time
        #print('slave node time')
        #print(data)
        
        print(datat)

        #origin segments
        print("number of segments : ",OriginSegments)
        #overlap
        print("partition : ",partition)
        #reduce time
        print('master_time : '+str(fftime-ftime))
        print('slave_time : '+str(ftime-stime))
        #whole time
        print('whole_time : '+str(fftime-stime))
            
        '''
        '''
        import matplotlib.pyplot as plt
        #save result
        fig = plt.figure("Superpixels -- %d segments" % (OriginSegments),dpi = 96,figsize = (img_w/96,img_h/96))
        ax = fig.add_subplot(1, 1, 1)
        
        
        
        ax.imshow(mark_boundaries(images, labels[0]))
        plt.xticks([]), plt.yticks([])
        plt.tight_layout()
        plt.subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, hspace = 0, wspace = 0)

        #plt.savefig('/home/gangmin/image-resource/result.png',bbox_inces='tight',pad_inches=0)

        plt.axis("off")

        # show the plots
        plt.show()
        '''
        
        '''
        df = pd.DataFrame(labels[0])
        df.to_csv('/home/gangmin/experiment/mrslic/exp2/'+str(OriginSegments)+'/'+folder+'/'+str(overlap)+'/'+img_name[:-4]+".csv",header=None,index=None)
        '''
        evaluation.append(fftime-stime)
        
    
    print(partition)
    print(np.mean(np.array(evaluation)))
                
